<a href="https://colab.research.google.com/github/themoonwalker1/quantcap-options/blob/main/options_data_retrieval.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Quant Cap Options Trading 23/24

In [ ]:
import requests
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import json

In [ ]:
class GatherData():
    """ Pulls EOD Options, EOD Greeks and Open Interest data from ThetaData and returns a dataframe """

    def get_historical_data(self):
        return


    def get_sentiment_data(self):
        """ Pulls data from StockGeist and returns a data frame """

        STOCKGEIST_API_KEY = ''
        headers  = { "token" : STOCKGEIST_API_KEY}

        # API endpoint
        base_url = "https://api.stockgeist.ai"
        asset_class = "stocks"  # or "crypto"
        location = "us"  # or "global"


        # General Parameters
        end_date = datetime.now().strftime("%Y-%m-%d")
        start_date = (datetime.now() - timedelta(days=30)).strftime("%Y-%m-%d")
        interval = "1d"
        symbol = "AAPL"  # example symbol

        # Messages sentiment data

        messages_url = f"{base_url}/{asset_class}/{location}/hist/message-metrics"
        messages_params = {
                          "symbol": symbol,
                          "start_date": start_date,
                          "end_date": end_date,
                          "interval": interval
                          }
        #....

        # Articles sentiment data

        articles_url = f"{base_url}/{asset_class}/{location}/hist/article-metrics"
        articles_params = {
                          "symbol": symbol,
                          "start_date": start_date,
                          "end_date": end_date,
                          "interval": interval
                          }
        #....




        return
    
    def get_fundamental_data(self):
        """ Pulls data from Tingo and returns a dataframe"""

        pass

    """ Pulls data from NASDAQ and returns a dataframe """

    def get_liquidity_data(self):
        pass
    """ Pulls data from Yahoo Calendar and returns a dataframe"""

    def get_earnings_data(self):
        pass
    """ Pulls data from 13F-Form Dataset and returns a dataframe"""

    def get_institutional_data(self):
        pass
    """ Parses through response and constructs and returns a dataframe"""

    def json_to_dtf(self):
        pass

In [ ]:


class ProcessData():
    def join_all_datasets(self, arr, methods):
        """
        joins all of the datasets in an array of dataframes using the
         specified methods in the string array of methods.
         Methods include full, left, right, inner"""
        pass

    """ handles missing NaN values using an approach passed in as a string variable named approach for dataset d. Approaches allowed include one-hot encoding, dropping all rows with NaN, etc."""

    def handle_missing_values(self, approach, dataset):
        pass
    """ Normalize features of dataframe x using sklearn methods using an approach passed in as a string variable approach"""

    def normalize_features(self, x, approach):
        pass

    """ Select target features of x, where the features are given in an array feature list"""

    def select_features(self, dataset, feature_list):
        pass
    """ Splits data into training, validation, and testing """

    def split_data(self, dataset, column, test_size, val_size):
        pass
    """ Deletes rows of data containing outliers above a certain threshold from dataset """

    def handle_outliers(self, dataset, threshold):
        pass

In [ ]:
print("testing")
print("hi")

testing
hi


In [ ]:
def fetch_smalltable_rows(
    table_handle: smalltable.Table,
    keys: Sequence[bytes | str],
    require_all_keys: bool = False,
) -> Mapping[bytes, tuple[str, ...]]:
    """Fetches rows from a Smalltable.

    Retrieves rows pertaining to the given keys from the Table instance
    represented by table_handle.  String keys will be UTF-8 encoded.

    Args:
        table_handle: An open smalltable.Table instance.
        keys: A sequence of strings representing the key of each table
          row to fetch.  String keys will be UTF-8 encoded.
        require_all_keys: If True only rows with values set for all keys will be
          returned.

    Returns:
        A dict mapping keys to the corresponding table row data
        fetched. Each row is represented as a tuple of strings. For
        example:

        {b'Serak': ('Rigel VII', 'Preparer'),
         b'Zim': ('Irk', 'Invader'),
         b'Lrrr': ('Omicron Persei 8', 'Emperor')}

        Returned keys are always bytes.  If a key from the keys argument is
        missing from the dictionary, then that row was not found in the
        table (and require_all_keys must have been False).

    Raises:
        IOError: An error occurred accessing the smalltable.
    """